In [1]:
import torch
import os
from tqdm import tqdm

In [4]:
base_path = "/home1/ohs/ongoing_proj/xtwice/cechallenge/v1_30b/pyllama/pyllama_data/30B"
pths = os.listdir(base_path)

ckpt_lst = []

for pth in tqdm(pths):
    if "consolidated" in pth:
        ckpt_path = os.path.join(base_path, pth)
        ckpt_lst.append(torch.load(ckpt_path, map_location="cpu"))

100%|██████████| 6/6 [00:37<00:00,  6.25s/it]


In [5]:
keys_lst = []
for i, ckpt in enumerate(ckpt_lst):
    keys_lst.append(list(ckpt.keys()))
print(len(keys_lst))

for i, keys in enumerate(keys_lst):
    for j, key_ in enumerate(keys):
        for k, keys_ in enumerate(keys_lst):
            if key_ not in keys_:
                print(i, j, k, key_)

4


In [6]:
def check_colpar(str):
    column_parallel_name_lst = ["output", "wq", "wk", "wv", "w1", "w3"]
    for colpar_name in column_parallel_name_lst:
        if colpar_name in str:
            return True
    return False

def check_rowpar(str):
    row_parallel_name_lst = ["embeddings", "wo", "w2"]
    for rowpar_name in row_parallel_name_lst:
        if rowpar_name in str:
            return True
    return False

def check_nopar(str):
    no_parallel_name_lst = ["rope", "norm"]
    for nopar_name in no_parallel_name_lst:
        if nopar_name in str:
            return True
    return False

In [7]:
model_weights_dict = {}
for i, model_weights_name in enumerate(tqdm(keys_lst[0])):
    
    a = None
    for j in range(3):
        
        if a is None:
            if check_colpar(model_weights_name):
                a = torch.concat((ckpt_lst[j][model_weights_name],
                                  ckpt_lst[j+1][model_weights_name]), dim=0)
            elif check_rowpar(model_weights_name):
                a = torch.concat((ckpt_lst[j][model_weights_name],
                                  ckpt_lst[j+1][model_weights_name]), dim=1)
            elif check_nopar(model_weights_name):
                a = ckpt_lst[j][model_weights_name]
                break
            else:
                assert True, "not matching"
        else:
            if check_colpar(model_weights_name):
                a = torch.concat((a,
                                  ckpt_lst[j+1][model_weights_name]), dim=0)
            elif check_rowpar(model_weights_name):
                a = torch.concat((a,
                                  ckpt_lst[j+1][model_weights_name]), dim=1)
            else:
                assert True, "not matching"
    
    model_weights_dict[model_weights_name] = a

len(model_weights_dict)
torch.save(model_weights_dict,"./whole_llama1_model.pt")

100%|██████████| 544/544 [00:27<00:00, 19.82it/s]


In [10]:
import torch

In [11]:
ckpt = torch.load("whole_llama1_model.pt")

In [12]:
for k,v in ckpt.items():
    print(k, v.shape)

tok_embeddings.weight torch.Size([32000, 6656])
norm.weight torch.Size([6656])
output.weight torch.Size([32000, 6656])
layers.0.attention.wq.weight torch.Size([6656, 6656])
layers.0.attention.wk.weight torch.Size([6656, 6656])
layers.0.attention.wv.weight torch.Size([6656, 6656])
layers.0.attention.wo.weight torch.Size([6656, 6656])
layers.0.feed_forward.w1.weight torch.Size([17920, 6656])
layers.0.feed_forward.w2.weight torch.Size([6656, 17920])
layers.0.feed_forward.w3.weight torch.Size([17920, 6656])
layers.0.attention_norm.weight torch.Size([6656])
layers.0.ffn_norm.weight torch.Size([6656])
layers.1.attention.wq.weight torch.Size([6656, 6656])
layers.1.attention.wk.weight torch.Size([6656, 6656])
layers.1.attention.wv.weight torch.Size([6656, 6656])
layers.1.attention.wo.weight torch.Size([6656, 6656])
layers.1.feed_forward.w1.weight torch.Size([17920, 6656])
layers.1.feed_forward.w2.weight torch.Size([6656, 17920])
layers.1.feed_forward.w3.weight torch.Size([17920, 6656])
layers.